In [1]:
import os
import sys
import numpy as np
import tifffile
import json
%matplotlib inline
import matplotlib.pyplot as plt

In [27]:
path = '/groups/spruston/sprustonlab/Boaz/20250321_ANM560898_No window/'
files = os.listdir(path)
files

['Day3_FRAP_00001_00001.tif',
 'Day3_morning_00001_00001.tif',
 'Day3_morning_00001_00002.tif',
 'Day3_morning_00001_00003.tif',
 'Day3_morning_00001_00004.tif',
 'Day3_morning_00001_00005.tif',
 'Day3_morning_00001_00006.tif',
 'Day3_morning_00001_00007.tif',
 'Day3_morning_00001_00008.tif',
 'Day3_morning_00001_00009.tif',
 'Day3_morning_00001_00010.tif']

In [28]:
def reshape_tiff_file(tiff_path, ops_path):
    """Reshape a mesoscope TIFF file based on Suite2p ops.json parameters, handling both 2D and 3D images."""

    # Load ops.json
    with open(ops_path, "r") as json_file:
        ops = json.load(json_file)

    # Load the original TIFF file
    with tifffile.TiffFile(tiff_path) as tif:
        images = tif.asarray()
    print(images.shape)
    # Check if image is 2D or 3D
    is_3d = len(images.shape) == 3  # True if (Z, Y, X)
    
    # Extract relevant ops parameters
    dx = np.array(ops["dx"])
    dy = np.array(ops["dy"])
    lines = ops["lines"]
    nrois = ops["nrois"]
    
    # Compute output image dimensions
    total_width = max(dx) + images.shape[-1]  # Width is determined by horizontal shifts
    total_height = max(dy) + max([len(line_range) for line_range in lines])  # Max height based on lines
    
    # Initialize the full FOV image
    if is_3d:
        num_planes = images.shape[0]
        full_fov = np.zeros((num_planes, total_height, total_width), dtype=images.dtype)
    else:
        full_fov = np.zeros((total_height, total_width), dtype=images.dtype)
    
    # Reassign pixel values based on ops.json parameters
    for i in range(nrois):
        x_offset = dx[i]
        y_offset = dy[i]
        line_range = lines[i]
        # Extract the corresponding ROI from the stacked input image
        if is_3d:
            source_section = images[:, line_range, :]
        else:
            source_section = images[line_range, :]
        
        # Place the ROI into the full FOV at the correct offsets
        if is_3d:
            full_fov[:, y_offset:y_offset + source_section.shape[1], x_offset:x_offset + source_section.shape[2]] = source_section
        else:
            full_fov[y_offset:y_offset + source_section.shape[0], x_offset:x_offset + source_section.shape[1]] = source_section
    
    return full_fov

In [29]:
import os
import re
import tifffile

def find_tiff_series(directory):
    """Finds TIFF files in a given directory that match the pattern [base_name_00001_00001.tif] and extracts the base name and number of images per file."""
    
    # Regular expression pattern to match TIFF files with numbered suffixes
    pattern = re.compile(r"(.+)_\d{5}\.tif$")
    
    tiff_info = {}
    
    for filename in os.listdir(directory):
        match = pattern.match(filename)
        if match:
            base_name = match.group(1)
            file_path = os.path.join(directory, filename)
            
            # Load TIFF metadata to get the number of images in each file
            with tifffile.TiffFile(file_path) as tif:
                num_images = len(tif.pages)
            
            # Store the base name and image count
            if base_name not in tiff_info:
                tiff_info[base_name] = []
            
            tiff_info[base_name].append((filename, num_images))
    
    return tiff_info


In [30]:
info =find_tiff_series(path)

In [31]:
info

{'Day3_FRAP_00001': [('Day3_FRAP_00001_00001.tif', 150)],
 'Day3_morning_00001': [('Day3_morning_00001_00001.tif', 101),
  ('Day3_morning_00001_00002.tif', 101),
  ('Day3_morning_00001_00003.tif', 101),
  ('Day3_morning_00001_00004.tif', 101),
  ('Day3_morning_00001_00005.tif', 101),
  ('Day3_morning_00001_00006.tif', 101),
  ('Day3_morning_00001_00007.tif', 101),
  ('Day3_morning_00001_00008.tif', 101),
  ('Day3_morning_00001_00009.tif', 101),
  ('Day3_morning_00001_00010.tif', 101)]}

In [ ]:
ops_files = {'10min_label_16hWash_00001':'/groups/spruston/sprustonlab/Boaz/20250320_ANM560898_No window/zStack.json',
             '10min_label_16hWash_2_00001':'/groups/spruston/sprustonlab/Boaz/20250320_ANM560898_No window/zStack.json',
             'Day2_FRAP_00001':'/groups/spruston/sprustonlab/Boaz/20250320_ANM560898_No window/frap.json',
             'Day2_FRAP_100Power_00001':'/groups/spruston/sprustonlab/Boaz/20250320_ANM560898_No window/frap.json',
             'Day2_FRAP_Post_00001':'/groups/spruston/sprustonlab/Boaz/20250320_ANM560898_No window/frap_post.json',
             'Day2_night_00001':'/groups/spruston/sprustonlab/Boaz/20250320_ANM560898_No window/zStack.json',

In [8]:
ops_path_32 = '/groups/spruston/sprustonlab/Boaz/20250319_ANM560898_No window/ops32.json'

In [ ]:
'/groups/spruston/sprustonlab/Boaz/202

In [9]:
ops_path

'/groups/spruston/sprustonlab/Boaz/20250319_ANM560898_No window/ops.json'

In [32]:
for base_name, file_list in info.items():
    if not base_name =='Day3_morning_00001':
        continue
    print(f"\nProcessing series: {base_name}")

    reshaped_images_list = []  # List to collect reshaped 3D arrays

    for filename, _ in file_list:
        file_path = os.path.join(path, filename)
        print(f"Opening file: {file_path}")

        # Call reshape_tiff_file function to process the images
        reshaped_image = reshape_tiff_file(file_path, '/groups/spruston/sprustonlab/Boaz/20250320_ANM560898_No window/zStack.json')


        reshaped_images_list.append(reshaped_image)

    # Convert list to NumPy array for proper 3D stacking
    reshaped_images_stack = np.concatenate(reshaped_images_list, axis=0)
    break


Processing series: Day3_morning_00001
Opening file: /groups/spruston/sprustonlab/Boaz/20250321_ANM560898_No window/Day3_morning_00001_00001.tif
(101, 7150, 478)
Opening file: /groups/spruston/sprustonlab/Boaz/20250321_ANM560898_No window/Day3_morning_00001_00002.tif
(101, 7150, 478)
Opening file: /groups/spruston/sprustonlab/Boaz/20250321_ANM560898_No window/Day3_morning_00001_00003.tif
(101, 7150, 478)
Opening file: /groups/spruston/sprustonlab/Boaz/20250321_ANM560898_No window/Day3_morning_00001_00004.tif
(101, 7150, 478)
Opening file: /groups/spruston/sprustonlab/Boaz/20250321_ANM560898_No window/Day3_morning_00001_00005.tif
(101, 7150, 478)
Opening file: /groups/spruston/sprustonlab/Boaz/20250321_ANM560898_No window/Day3_morning_00001_00006.tif
(101, 7150, 478)
Opening file: /groups/spruston/sprustonlab/Boaz/20250321_ANM560898_No window/Day3_morning_00001_00007.tif
(101, 7150, 478)
Opening file: /groups/spruston/sprustonlab/Boaz/20250321_ANM560898_No window/Day3_morning_00001_0000

In [54]:
reshaped_images_stack.shape

(100, 1768, 1912)

In [33]:
ind =  np.arange(100, 1010, 101)

In [34]:
images_filtered = np.delete(reshaped_images_stack, ind, axis=0)

In [35]:
images_filtered.shape

(1000, 1768, 1912)

In [36]:
output = path + base_name + '_resize.tif'

In [37]:
output

'/groups/spruston/sprustonlab/Boaz/20250321_ANM560898_No window/Day3_morning_00001_resize.tif'

In [38]:
tifffile.imwrite(output, images_filtered)

In [24]:
tifffile.imwrite(output, reshaped_images_stack)

In [40]:
tifffile.imwrite(output[:-4] + 'max2.tif', images_filtered.max(1))

In [39]:
tifffile.imwrite(output[:-4] + 'max1.tif', images_filtered.max(0))

In [25]:
del reshaped_images_stack

In [26]:
del images_filtered